In [450]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import cross_validation
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot    
import plotly.graph_objs as go
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from sklearn.cross_validation import train_test_split
import keras
from random import randint

import random

# --- GENSIM PACKAGE ---
import gensim, logging
from gensim.models import Word2Vec, doc2vec, Doc2Vec

from gensim.models import doc2vec
from collections import namedtuple


%matplotlib inline


### function to get doc2vec

In [175]:
def doc_to_vec(doc):
    doc1 = []
    doc1.append(doc)
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
    for i, text in enumerate(doc1):
        words = text.lower().split()
        tags = [i]
        docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)

    model = doc2vec.Doc2Vec(docs, size = 200, window = 300, min_count = 1, workers = 4)

    # Get the vectors

    s = model.docvecs
    return s

### loading data

In [22]:
train_v1 = pd.read_json('../data/train-v1.1.json')
test_v1 = pd.read_json('../data/dev-v1.1.json')
squad_train = pd.read_json('../data/squad_train_doc.json')
squad_test = pd.read_json('../data/squad_dev_doc.json')

In [24]:
data_train = squad_train
data_train.rename(columns={'passages': 'documents'}, inplace=True)


In [25]:
data_train.head()

,documents,title
0,[{'questions': ['To whom did the Virgin Mary a...,University_of_Notre_Dame
1,[{'questions': ['In what city did Beyonce grow...,Beyoncé
2,[{'questions': ['How many ranges are part of t...,Montana
3,[{'questions': ['What 2001 case was declared g...,Genocide
4,[{'questions': ['What does emergence of resist...,Antibiotics


### Document list

In [26]:
title_list = [data_train.title[i] for i in range(data_train.shape[0])]

### Segregating Dataframe

In [27]:
## Returns list of all context and list of all question sets

def get_compact_dataframe():
    context_list = []
    question_list = []
    
    for doc in data_train.documents:
        context_list.append(get_each_context_each_questionSet(doc)[0])
        question_list.append(get_each_context_each_questionSet(doc)[1])
    
    return context_list, question_list

In [28]:
## Pass a document and it returns list of context and list of set of questions

def get_each_context_each_questionSet(document):
    each_doc_context_list = [document[i]['context'] for i in range(len(document))]
    each_doc_question_list = [document[i]['questions'] for i in range(len(document))]
    return  each_doc_context_list, each_doc_question_list

In [29]:
context_list, question_list = get_compact_dataframe()[0], get_compact_dataframe()[1]
compact_dataframe = pd.DataFrame({'title':title_list, 'context':context_list, 'questions': question_list})

## DATAFRAME

In [30]:
compact_dataframe.head()

,context,questions,title
0,"[Architecturally, the school has a Catholic ch...",[[To whom did the Virgin Mary allegedly appear...,University_of_Notre_Dame
1,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,"[[In what city did Beyonce grow up?, In what R...",Beyoncé
2,[Montana i/mɒnˈtænə/ is a state in the Western...,[[How many ranges are part of the Rocky Mounta...,Montana
3,"[The phrase ""in whole or in part"" has been sub...",[[What 2001 case was declared genocide by the ...,Genocide
4,[The emergence of resistance of bacteria to an...,"[[What does emergence of resistance reflect?, ...",Antibiotics


In [31]:
compact_dataframe[compact_dataframe.title == '2008_Sichuan_earthquake']

,context,questions,title
10,[The 2008 Sichuan earthquake or the Great Sich...,"[[What was the earthquake named?, What did the...",2008_Sichuan_earthquake


### Total questions

In [32]:
list_questions_perDOC = []
count = 0

for each_doc_question in compact_dataframe.questions:

    Doc_questions = []
    for each_Set in each_doc_question:
        for each_question in each_Set:
            Doc_questions.append(each_question)
            count +=1
    
    list_questions_perDOC.append(Doc_questions)

### Implementing Gensim 

In [35]:
class DocIterator(object):
    
    ## Initailizes document's list(doc1,doc2...) and its label's list('Book_1','Book_2',...)
    def __init__(self, doc_list, labels_list):
        self.doc_list = doc_list
        self.labels_list = labels_list
    
    ## Assigns label1 to a list of all words in doc1, label2 to all words in doc2, etc.
    def __iter__(self):
        
        for idx, doc in enumerate(self.doc_list):
            yield doc2vec.LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])

In [36]:
## Contains the whole data

data = []
docLabels = title_list

for i in docLabels:
    x = compact_dataframe[compact_dataframe.title == i].context.sum()
    y = ' '.join(x)
    data.append(y)

In [37]:
iter_docs = DocIterator(data, docLabels)

### Model

In [38]:
model = Doc2Vec(iter_docs, size=200, window=50, min_count=1, workers=11, alpha=0.025, min_alpha=0.025, dbow_words = 1, iter = 20)

### creating dummy dataframe using five questions from 5 documents

In [44]:
questions_list = ['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?','In what city did Beyonce grow up?',
                 'How many ranges are part of the Rocky Mountains?',
                  'What 2001 case was declared genocide by the International Criminal Tribune for the Former Yugoslavia (ICTY)?',
                 'What does emergence of resistance reflect?']

In [64]:
documents_doc2vec = []
for i in range(5):
    documents_doc2vec.append(model.docvecs[i])


In [123]:
data_d = []
for i in range(5):
    for j in range(5):
        data_d.append(title_list[j])  

In [124]:
data_q = []
for i in range(5):
    for j in range(5):
        data_q.append(questions_list[i])

In [125]:
def scores():
    scores = []
    for i in range(25):
        scores.append(random.random())
    return scores    

In [126]:
s1 = scores()
s2 = scores()
s3 = scores()
s4 = scores()

In [127]:
print(randint(2,25))

20


In [139]:
random.sample(range(5), 1)

[2]

In [157]:
y =[]
rank = []
for i in range(5):
    k = 2
    for j in range(5):
        if(i==j):
            y.append(1)
            rank.append(1)
        else:
            y.append(0)
            rank.append(k)
            k += 1
rank

[1, 2, 3, 4, 5, 2, 1, 3, 4, 5, 2, 3, 1, 4, 5, 2, 3, 4, 1, 5, 2, 3, 4, 5, 1]

In [158]:
q_vs_d = pd.DataFrame(
    {'questions': data_q,
     'documents': data_d,
     's1': s1,
     's2': s2,
     's3': s3,
     's4': s4,
     'y': y,
     'rank':rank
    })

In [ ]:
q_vs_d

In [308]:
q_vs_d

,documents,questions,rank,s1,s2,s3,s4,y
0,University_of_Notre_Dame,To whom did the Virgin Mary allegedly appear i...,1,0.033688,0.980017,0.732910,0.344520,1
1,Beyoncé,To whom did the Virgin Mary allegedly appear i...,2,0.034995,0.053208,0.037961,0.349009,0
2,Montana,To whom did the Virgin Mary allegedly appear i...,3,0.229629,0.199111,0.524838,0.636342,0
3,Genocide,To whom did the Virgin Mary allegedly appear i...,4,0.023771,0.383044,0.529787,0.272755,0
4,Antibiotics,To whom did the Virgin Mary allegedly appear i...,5,0.883591,0.318003,0.605177,0.360128,0
5,University_of_Notre_Dame,In what city did Beyonce grow up?,2,0.667515,0.212016,0.827821,0.241444,0
6,Beyoncé,In what city did Beyonce grow up?,1,0.556341,0.300489,0.902925,0.569973,1
7,Montana,In what city did Beyonce grow up?,3,0.429982,0.200030,0.642687,0.640343,0
8,Genocide,In what city did Beyonce grow up?,4,0.685385,0.221075,0.569412,0.731313,0
9,Antibiotics,In what city did Beyonce grow up?,5,0.581354,0.655563,0.129616,0.863337,0


In [314]:
def input(question):
    df1 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == 1))]
    df2 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == max(rank)))]  
    df3 = q_vs_d[((q_vs_d['questions'] == question)& (q_vs_d['rank'] == randint(2,4)))] 
    new_df = df1.append(df2, ignore_index=True)
    new_df2 =new_df.append(df3, ignore_index=True)
    for i in range(3):
        new_df2['questions'][i] = doc_to_vec(question)[0]
        new_df2['documents'][i] = model.docvecs[title_list.index(new_df2['documents'][i])]
    y_train = new_df2['y']
    x_train2 = new_df2[['s1','s2','s3','s4']] 
    new_df2.drop(['y','rank','s1','s2','s3','s4'],axis = 1, inplace = True)
    x_train = new_df2
    return x_train,y_train,x_train2
    

In [408]:
x_train,y_train,x_train2 = input('To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?')

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/user/classification/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [409]:
x_train

,documents,questions
0,"[6.17416, -2.46013, -4.98347, -0.958448, -10.4...","[-0.000606836, -0.00185484, 0.000355614, 0.001..."
1,"[-0.744325, 1.94375, -9.40504, -8.04373, -6.64...","[-0.000606836, -0.00185484, 0.000355614, 0.001..."
2,"[-2.57714, 2.51253, -2.02251, -4.12971, -4.442...","[-0.000606836, -0.00185484, 0.000355614, 0.001..."


In [410]:
x_train2

,s1,s2,s3,s4
0,0.033688,0.980017,0.732910,0.344520
1,0.883591,0.318003,0.605177,0.360128
2,0.023771,0.383044,0.529787,0.272755


In [411]:
ytrain = y_train.values.tolist()
ytrain = keras.utils.to_categorical(ytrain, 2)



In [474]:
xtrain.shape

(3, 2)

In [413]:
xtrain = np.array(x_train)
xtrain2 = np.array(x_train2)

In [465]:
type(xtrain)

numpy.ndarray

In [471]:
xtrain.reshape(3,2,1)

array([[[ array([  6.17415524,  -2.46012878,  -4.98346853,  -0.95844787,
       -10.43505573,   0.02398554,  -0.04041549,   5.43331194,
         0.80976433,   1.66731644,  -3.58818197,  12.64922047,
        -1.46409917,  -4.81541252,   3.89282322,   5.08968496,
        -2.61389136,  -2.98656678,   4.45836926,   0.66334504,
         6.33978271,  -0.61466283,  -3.68178821,   0.61297858,
        -2.49723768,  -1.33421552,  -3.54099441,   3.41979456,
        -1.341295  ,  -0.12873079,  -3.2723403 ,  -9.25423813,
        -4.56694841,   2.47581601,   3.91783619,   0.8956762 ,
        -2.31236815,   3.89663887,   4.43468714,   1.80397689,
        -0.80405617,   2.29086709,   0.04693424,   4.26663113,
         4.814116  ,   2.2953918 ,   1.19473279,   3.20432758,
        -1.38736439,   1.27813399,   0.86190182,  -1.52637827,
        -2.03194523,   2.14347076,   5.57339954,   0.54350883,
         3.38212085,   2.14371037,   2.55167127,  -2.74619317,
        -6.00193787,  -1.24737966,  -2.733868

In [472]:
xtrain.shape

(3, 2)

In [436]:
l = []
for i in range(3):
    l1 = []
    for j in range(2):
        for k in xtrain[i][j]:
            l1.append(k)
    l.append(l1)
            

In [441]:
qd_array = np.array(l)

In [443]:
qd_array.shape[1]

400

200

In [483]:
model_scores = Sequential()
model_scores.add(Dense(600, activation='relu', input_shape=(200,3)))
model_scores.add(Dense(500, activation='relu'))

model_scores.add(Dense(ytrain.shape[1], activation='softmax'))
model_scores.compile(optimizer=RMSprop(),loss='categorical_crossentropy',metrics = ['accuracy'])
model_scores = model_scores.fit(xtrain, ytrain, batch_size=1, epochs=5, validation_split = .2, verbose=2)

ValueError: Error when checking model input: expected dense_118_input to have 3 dimensions, but got array with shape (3, 2)